In [1]:
from tabulate import tabulate

In [7]:
import wandb
import pandas as pd
import numpy as np

api = wandb.Api()

# Specify your project
project = "oualidzari/Link Inference In FL"

# Define your filters
filters = {
    "config.dataset": "Cora",
    "config.experiment_comment": "Lapgraph"
}

# Fetch filtered runs
runs = api.runs(project, filters=filters)



In [8]:
data = []
for run in runs:
    name = run.name
    epsilon = run.config.get("epsilon")

    # Fetch history
    history = run.history()
    accuracy_gradient = (
        history["Accuracy-gradients"].iloc[0]
        if "Accuracy-gradients" in history.columns
        else None
    )
    accuracy_labels = (
        history["Accuracy-labels"].iloc[0]
        if "Accuracy-labels" in history.columns
        else None
    )
    test_accuracy = (
        history["accuracy_test"].iloc[299]
        if "accuracy_test" in history.columns
        else None
    )

    data.append(
        {
            "name": name,
            "epsilon": epsilon,
            "acc_grad": accuracy_gradient,
            "acc_lab": accuracy_labels,
            "test_acc": test_accuracy,
        }
    )

In [9]:
df = pd.DataFrame(data)

In [10]:
runs[0].history()

,_step,_timestamp,AUC-forward_values,accuracy_test,Accuracy-features,epoch,accuracy_train,_runtime,Accuracy-forward_values,Accuracy-gradients,Accuracy-labels,AUC-features,Accuracy-output_server,loss,AUC-output_server,AUC-gradients
0,0,1.716941e+09,0.557471,0.086923,0.627651,0,0.076071,7.740663,0.560072,0.807020,0.815614,0.645422,0.599577,1.946974,0.565758,0.805010
1,1,1.716941e+09,0.567610,0.087692,NaN,1,0.076809,8.686803,0.537773,0.795425,NaN,NaN,0.606648,1.922544,0.580919,0.802203
2,2,1.716941e+09,0.606264,0.279231,NaN,2,0.320532,9.537115,0.566372,0.802625,NaN,NaN,0.567028,1.898561,0.580178,0.794105
3,3,1.716941e+09,0.619297,0.279231,NaN,3,0.320532,10.294979,0.604293,0.814904,NaN,NaN,0.564216,1.869580,0.572077,0.794695
4,4,1.716941e+09,0.615092,0.279231,NaN,4,0.320532,11.159193,0.578811,0.816985,NaN,NaN,0.544235,1.834411,0.551695,0.796400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,1.716941e+09,0.583728,0.814615,NaN,295,1.000000,261.996674,0.560957,0.472115,NaN,NaN,0.796033,0.009119,0.873802,0.632907
296,296,1.716941e+09,0.583590,0.818462,NaN,296,1.000000,262.838732,0.564629,0.503941,NaN,NaN,0.792758,0.009093,0.868373,0.627055
297,297,1.716941e+09,0.583795,0.813846,NaN,297,1.000000,263.676831,0.543975,0.474443,NaN,NaN,0.794736,0.008823,0.873658,0.632031
298,298,1.716941e+09,0.583822,0.820000,NaN,298,1.000000,264.474197,0.549029,0.496630,NaN,NaN,0.790515,0.008368,0.870293,0.633099


In [16]:
# Group by budget and calculate statistics
grouped_stats = df.groupby('epsilon').agg({
    'acc_grad': ['mean', 'std'],
    'acc_lab': ['mean', 'std'],
    'test_acc': ['mean', 'std']
}).sort_values(by='epsilon', ascending=False)

# Function to format mean and std as percentages
def format_mean_std(row):
    return f"{row['mean']*100:.2f} ± {row['std']*100:.2f}"

# Apply formatting
formatted_stats = pd.DataFrame()
for metric in ['acc_grad', 'acc_lab', 'test_acc']:
    formatted_stats[metric] = grouped_stats[metric].apply(format_mean_std, axis=1)

# Reset index to make budget a column and convert to percentage
formatted_stats = formatted_stats.reset_index()
# formatted_stats['budget'] = (formatted_stats['budget'] * 100).astype(int)

# Create a formatted table without row dividers
table = tabulate(formatted_stats, headers=['epsilon', 'acc_grad', 'acc_lab', 'test_acc'], 
                 tablefmt='simple', showindex=False)

print(table)

  epsilon  acc_grad      acc_lab       test_acc
---------  ------------  ------------  ------------
       10  81.57 ± 0.89  81.56 ± 0.33  81.41 ± 0.64
        9  81.46 ± 0.16  81.56 ± 0.33  79.08 ± 0.50
        8  81.67 ± 0.30  81.56 ± 0.33  75.38 ± 0.73
        7  81.46 ± 0.19  81.56 ± 0.33  65.23 ± 0.95
        6  81.43 ± 0.52  81.56 ± 0.33  55.97 ± 0.69
        5  81.33 ± 0.15  81.56 ± 0.33  49.23 ± 1.12
        4  81.30 ± 0.23  81.56 ± 0.33  46.56 ± 0.25
        3  81.15 ± 0.49  81.56 ± 0.33  42.05 ± 5.24
        2  81.39 ± 0.28  81.56 ± 0.33  44.95 ± 1.41
        1  81.30 ± 0.24  81.56 ± 0.33  45.56 ± 0.96
       -1  81.32 ± 0.66  81.56 ± 0.33  83.97 ± 0.75
